<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_tabelog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.2ctY19yaVh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Jg75rBfnAQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.j3Yrmx2iu5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
#環境変数設定
TARGET_URL = 'https://r.gnavi.co.jp/saitama/#link-searchShops'
DRIVER_WAIT_TIME = 2

#取得データ格納先
result_dict = { 
    '店名': [],
    '電話番号':[],
    '住所': [],
    'URL':[],
}

In [ ]:
#標準ライブラリ
import os
import json

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config
)
# drive.mount('/content/drive')

2023-04-13 14:46:17,090 [INFO] numexpr.utils: NumExpr defaulting to 2 threads.


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [ ]:
root = 'https://r.gnavi.co.jp/saitama/#link-searchShops'
target = '埼玉県'

#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(root)
      logger.debug(f'{target}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{target}：トップページへ移動出来ませんでした: [e]')

2023-04-13 14:46:40,211 [DEBUG] main: 埼玉県：トップページへ移動しました


In [ ]:
#ジャンル別URL取得
a_elements = driver.find_elements(By.CSS_SELECTOR,".swiper-wrapper a")
target_urls = [a_element.get_attribute('href') for a_element in a_elements]
print(target_urls)

['https://r.gnavi.co.jp/area/saitama/kods00317/rs/', 'https://r.gnavi.co.jp/plan/campaign?sc_lid=cp_preftop_mainPickup_01', 'https://r.gnavi.co.jp/area/saitama/kods00317/rs/', 'https://r.gnavi.co.jp/plan/campaign?sc_lid=cp_preftop_mainPickup_01', 'https://pr.gnavi.co.jp/mb/gurunavi-select/?area=PREF11&sc_lid=mb_saitama_preftop', 'https://r.gnavi.co.jp/area/saitama/kods00066/rs/', 'https://r.gnavi.co.jp/area/saitama/kods00078/rs/', 'https://r.gnavi.co.jp/area/saitama/kods00145/rs/', 'https://r.gnavi.co.jp/kggmt72z0000/', 'https://r.gnavi.co.jp/gf61x3b80000/', 'https://r.gnavi.co.jp/1aac7zpn0000/', 'https://r.gnavi.co.jp/fru64z9v0000/', 'https://r.gnavi.co.jp/mau236h80000/', 'https://r.gnavi.co.jp/ffczvukw0000/', 'https://r.gnavi.co.jp/j127z0px0000/', 'https://r.gnavi.co.jp/gd7m901/', 'https://r.gnavi.co.jp/6nv2bve30000/', 'https://r.gnavi.co.jp/cwur2dhd0000/', 'https://r.gnavi.co.jp/87nevypn0000/', 'https://r.gnavi.co.jp/c1g7fzh50000/', 'https://r.gnavi.co.jp/e969100/', 'https://r.gnavi

In [ ]:
#ページ遷移(トップページ → 店舗リストページ)
target_url = target_urls[0]
try:
      driver.get(target_url)
      logger.debug(f'{target}：店舗リストページへ移動しました')
except Exception as e:
      logger.error(f'{target}：店舗リストページへ移動出来ませんでした: [e]')

2023-04-13 14:47:51,664 [DEBUG] main: 埼玉県：店舗リストページへ移動しました


In [ ]:
#店舗リスト全ページ取得

#各店舗URLを取得
a_elements = driver.find_elements(By.CSS_SELECTOR,".style_title___HrjW a")
store_urls = [a_element.get_attribute('href') for a_element in a_elements]
print(len(store_urls))

24


In [ ]:
#店舗個別URLへ遷移して情報取得

#既に取得済みの店舗はスキップ

for store_url in store_urls:
  try:
      driver.get(store_url)
      logger.debug(f'{target}：店舗リストページへ移動しました')
  except Exception as e:
      logger.error(f'{target}：店舗リストページへ移動出来ませんでした: [e]')

  #店舗名/住所/電話番号取得
  store_name = driver.find_element(By.ID,"info-name").text
  phone_number = driver.find_element(By.CSS_SELECTOR,".number").text 
  address = driver.find_element(By.CSS_SELECTOR,".region").text
  
  print(store_name)

  #結果格納
  result_dict['店名'].append(store_name)
  result_dict['電話番号'].append(phone_number)
  result_dict['住所'].append(address)
  result_dict['URL'].append(store_url)
  
df_result = pd.DataFrame(result_dict)

df_result

2023-04-13 15:03:03,794 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
SOMETHING サムシング 浦和本店
2023-04-13 15:03:10,146 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
全席個室 楽蔵‐RAKUZO‐ 大宮東口駅前店
2023-04-13 15:03:15,824 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
和食 鶏料理 けんび
2023-04-13 15:03:20,090 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
ロイヤルホスト 浦和南店
2023-04-13 15:03:24,010 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
【１日２組限定完全予約制】 古民家レストラン 味市場のはなれ
2023-04-13 15:03:31,268 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
川越焼肉 Kan
2023-04-13 15:03:36,669 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
銀座ライオン 川口店
2023-04-13 15:03:41,889 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
イザカヤ
2023-04-13 15:03:47,035 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
鉄板焼dining 炉庵 ‐roan‐
2023-04-13 15:03:52,435 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
しゃらく 戸田公園店
2023-04-13 15:03:56,708 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
おでん家 万昌 ぶらっせ 大宮本店
2023-04-13 15:04:01,487 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
チーズラボ7号店
2023-04-13 15:04:05,737 [DEBUG] main: 埼玉県：店舗リストページへ移動しました
千の庭 川口店
2023-04-13 15:04:10,577 [DEBUG] main: 埼玉県：店舗リストページへ移動しま

,店名,電話番号,住所,URL
0,[<selenium.webdriver.remote.webelement.WebElem...,[<selenium.webdriver.remote.webelement.WebElem...,[<selenium.webdriver.remote.webelement.WebElem...,https://r.gnavi.co.jp/e363413/?sc_type=area&sc...
1,[<selenium.webdriver.remote.webelement.WebElem...,[<selenium.webdriver.remote.webelement.WebElem...,[<selenium.webdriver.remote.webelement.WebElem...,https://r.gnavi.co.jp/e363413/?sc_type=area&sc...
2,[],[],[],https://r.gnavi.co.jp/plan/e363413/plan-reserv...
3,[],[],[],https://r.gnavi.co.jp/plan/e363413/plan-reserv...
4,[],[],[],https://r.gnavi.co.jp/plan/e363413/plan-reserv...
...,...,...,...,...
429,焼肉万里 武蔵浦和店,050-5487-1338,埼玉県さいたま市南区別所7-7-7,https://r.gnavi.co.jp/df7ak1dz0000/
430,五代目 蔵ＤＩＮＩＮＧ 酒田屋商店,050-5484-8125,埼玉県飯能市本町4-4,https://r.gnavi.co.jp/a879203/
431,しゃぶしゃぶどん亭 上尾店,050-5493-4572,埼玉県上尾市上町2-5-34,https://r.gnavi.co.jp/kkwauegj0000/
432,博多もつ鍋酒場 金子増太郎,050-5487-6795,埼玉県川越市新富町2-13-5,https://r.gnavi.co.jp/p442500/


In [ ]:
#埼玉県　大宮

#お店一覧
https://r.gnavi.co.jp/area/aream2402/rs/
https://r.gnavi.co.jp/area/aream2402/rs/?p=2

#個別店舗
https://r.gnavi.co.jp/s35gcc750000/?sc_type=area&sc_area=AREAM2402&sc_dsp=rs_138785

SyntaxError: ignored